In [2]:
import sys,os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
#from pathlib import Path

from ctypes import *
import math
#cv2_root = "/usr/local/lib-bk/python2.7/dist-packages/cv2"
#sys.path.insert(0,cv2_root)
#cv2_lib_avcodec = "/usr/lib/x86_64-linux-gnu/bkavcodec"
#sys.path.insert(0,cv2_lib_avcodec)
val_txt_file = "/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/2007_val_plate.txt"
time_str = (datetime.datetime.now()).strftime("%Y%m%d")
f = open(val_txt_file, 'r')
data = f.readlines()
seed_xml_names = []
GENE_XML_DIR = '/media/yyp/DATA/EVALUATE/' + time_str + '_detection_result/'
#GENE_XML_DIR = '/data/darknet/python/' + time_str + '_detection_result_864/'
for line in data:
    #print line
    xml_name = line.replace('JPEGImages', 'Annotations')
    xml_name = line.replace('.jpg', '.xml')
    seed_xml_names.append(xml_name)
    
'''
data_file = '/data/darknet/cfg/coco.data'
cfg_file = '/data/darknet/cfg/yolov3-nob-288.cfg'
weights_file = '/data/weights/header_int_aaa_yolov3.weights'
'''

#weights_file = '/data/darknet/weights/yolov3-tiny-ncs-without-last-maxpool-refined-anchors_120000.weights'
seed_xml_names.sort()

In [3]:
def sample(probs):
    s = sum(probs)
    probs = [a/s for a in probs]
    r = random.uniform(0, 1)
    for i in range(len(probs)):
        r = r - probs[i]
        if r <= 0:
            return i
    return len(probs)-1

def c_array(ctype, values):
    arr = (ctype*len(values))()
    arr[:] = values
    return arr

class BOX(Structure):
    _fields_ = [("x", c_float),
                ("y", c_float),
                ("w", c_float),
                ("h", c_float)]

class DETECTION(Structure):
    _fields_ = [("bbox", BOX),
                ("classes", c_int),
                ("prob", POINTER(c_float)),
                ("mask", POINTER(c_float)),
                ("objectness", c_float),
                ("sort_class", c_int)]


class IMAGE(Structure):
    _fields_ = [("w", c_int),
                ("h", c_int),
                ("c", c_int),
                ("data", POINTER(c_float))]

class METADATA(Structure):
    _fields_ = [("classes", c_int),
                ("names", POINTER(c_char_p))]
    
def classify(net, meta, im):
    out = predict_image(net, im)
    res = []
    for i in range(meta.classes):
        res.append((meta.names[i], out[i]))
    res = sorted(res, key=lambda x: -x[1])
    return res

def detect(net, meta, image, thresh=.5, hier_thresh=.5, nms=.45):
    im = load_image(image, 0, 0)
    num = c_int(0)
    pnum = pointer(num)
    predict_image(net, im)
    dets = get_network_boxes(net, im.w, im.h, thresh, hier_thresh, None, 0, pnum)
    num = pnum[0]
    if (nms): do_nms_obj(dets, num, meta.classes, nms);

    res = []
    for j in range(num):
        for i in range(meta.classes):
            if dets[j].prob[i] > 0:
                b = dets[j].bbox
                res.append((meta.names[i], dets[j].prob[i], (b.x, b.y, b.w, b.h)))
    res = sorted(res, key=lambda x: -x[1])
    free_image(im)
    free_detections(dets, num)
    return res
#lib = CDLL("/home/pjreddie/documents/darknet/libdarknet.so", RTLD_GLOBAL)
lib = CDLL("/data/darknet/libdarknet.so", RTLD_GLOBAL)
lib.network_width.argtypes = [c_void_p]
lib.network_width.restype = c_int
lib.network_height.argtypes = [c_void_p]
lib.network_height.restype = c_int

predict = lib.network_predict
predict.argtypes = [c_void_p, POINTER(c_float)]
predict.restype = POINTER(c_float)

set_gpu = lib.cuda_set_device
set_gpu.argtypes = [c_int]

make_image = lib.make_image
make_image.argtypes = [c_int, c_int, c_int]
make_image.restype = IMAGE

get_network_boxes = lib.get_network_boxes
get_network_boxes.argtypes = [c_void_p, c_int, c_int, c_float, c_float, POINTER(c_int), c_int, POINTER(c_int)]
get_network_boxes.restype = POINTER(DETECTION)

make_network_boxes = lib.make_network_boxes
make_network_boxes.argtypes = [c_void_p]
make_network_boxes.restype = POINTER(DETECTION)

free_detections = lib.free_detections
free_detections.argtypes = [POINTER(DETECTION), c_int]

free_ptrs = lib.free_ptrs
free_ptrs.argtypes = [POINTER(c_void_p), c_int]

network_predict = lib.network_predict
network_predict.argtypes = [c_void_p, POINTER(c_float)]

reset_rnn = lib.reset_rnn
reset_rnn.argtypes = [c_void_p]

load_net = lib.load_network
load_net.argtypes = [c_char_p, c_char_p, c_int]
load_net.restype = c_void_p

do_nms_obj = lib.do_nms_obj
do_nms_obj.argtypes = [POINTER(DETECTION), c_int, c_int, c_float]

do_nms_sort = lib.do_nms_sort
do_nms_sort.argtypes = [POINTER(DETECTION), c_int, c_int, c_float]

free_image = lib.free_image
free_image.argtypes = [IMAGE]

letterbox_image = lib.letterbox_image
letterbox_image.argtypes = [IMAGE, c_int, c_int]
letterbox_image.restype = IMAGE

load_meta = lib.get_metadata
lib.get_metadata.argtypes = [c_char_p]
lib.get_metadata.restype = METADATA

load_image = lib.load_image_color
load_image.argtypes = [c_char_p, c_int, c_int]
load_image.restype = IMAGE

rgbgr_image = lib.rgbgr_image
rgbgr_image.argtypes = [IMAGE]

predict_image = lib.network_predict_image
predict_image.argtypes = [c_void_p, IMAGE]
predict_image.restype = POINTER(c_float)

In [ ]:
#data_file = '/data/darknet/cfg/voc-veichle.data'
data_file = '/data/darknet/cfg/voc-vehicle-person-plate.data'
#data_file = '/data/darknet/cfg/coco.data'

all_wieghts_dir = '/media/yyp/DATA/EVALUATE/'
weights = os.listdir(all_wieghts_dir)
weights.sort()
print(weights)
for weight in weights:
    print(weight)
    if(not weight.endswith('.weights')):
        continue
    cfg_file = all_wieghts_dir + weight[:-len(weight.split('_')[-1])] + 'test.cfg'
    print(cfg_file)
    generated_xml_dir = GENE_XML_DIR + weight.split('.weights')[0] + '/'
    #if(os.path.exists(generated_xml_dir)):
    #   continue
    #os.system('rm -rf ' + GENE_XML_DIR)
    os.system('mkdir -p ' + generated_xml_dir)
    weights_file = all_wieghts_dir + weight
    net = load_net(cfg_file, weights_file, 0)
    meta = load_meta(data_file)
    for image_file in data:
        image = image_file.split('/')[-1]
        image = image[:-1]
        #if(os.path.exists(generated_xml_dir + image[:-4] + '.xml')):
        #    continue
        image_file = image_file[:-1]
        ret = image_file.endswith('.jpg')
        if ret == False:
            print('+++++++++++')
            continue
        #print('detect')
        r = detect(net, meta, image_file, 0.3, 0.5, 0.45)
        #print(len(r))

        origimg = cv2.imread(image_file)
        height = origimg.shape[0]
        width = origimg.shape[1]
        depth = origimg.shape[2]

        xml_str = "<annotation>\n\t\
        <folder>image</folder>\n\t\
        <filename>" + image + "</filename>\n\t\
        " + "<path>" + image_file + "</path>\n\t\
        <source>\n\t\t\
        <database>Unknown</database>\n\t\
        </source>\n\t\
        <size>\n\t\t\
        <width>" + str(width) + "</width>\n\t\t\
        <height>" + str(height) + "</height>\n\t\t\
        <depth>" + str(depth) + "</depth>\n\t\
        </size>\n\t\
        <segmented>0</segmented>"
        #print(xml_str)
        found_flag = False
        for i in range(len(r)):
            cls = r[i][0]
            #print(cls)
            if(cls not in ['plate', 'bus','car','truck', 'motorbike','bicycle','person', 'B', 'C', 'T']):
            #if(cls not in ['person', 'motorbike','bicycle']):
                continue
            found_flag = True
            score = r[i][1]
            bbox = r[i][2]

            x1 = max(0, (bbox[0] - bbox[2]/2))
            y1 = max(0, (bbox[1] - bbox[3]/2))
            x2 = min(width, (bbox[0] + bbox[2]/2))
            y2 = min(height, (bbox[1] + bbox[3]/2))      

            p1 = (int(x1), int(y1))
            p2 = (int(x2), int(y2))

            obj_str = "\n\t\
            <object>\n\t\t\
            <name>" + cls + "</name>\n\t\t\
            <pose>Unspecified</pose>\n\t\t\
            <truncated>0</truncated>\n\t\t\
            <difficult>0</difficult>\n\t\t\
            <bndbox>\n\t\t\t\
            <xmin>" + str(int(x1)) + "</xmin>\n\t\t\t\
            <ymin>" + str(int(y1)) + "</ymin>\n\t\t\t\
            <xmax>" + str(int(x2)) + "</xmax>\n\t\t\t\
            <ymax>" + str(int(y2)) + "</ymax>\n\t\t\
            </bndbox>\n\t\
            </object>"

            xml_str += obj_str

            cv2.rectangle(origimg, p1, p2, (0,255,0))
            p3 = (max(p1[0], 15), max(p1[1], 15))
            title = "%s:%.2f" % (cls, score)
            cv2.putText(origimg, title, p3, cv2.FONT_ITALIC, 0.6, (0, 255, 0), 1)
        xml_str += "\n</annotation>"
        #print('found_flag {0}'.format(found_flag))
        if (not found_flag):
            continue
        fileObject = open(generated_xml_dir + image[:-4] + '.xml', 'w')  
        fileObject.write(xml_str)
        fileObject.close()
        #cv2.imwrite('/data/'+ image_file.split('/')[-1], origimg)
        print("Created : " + generated_xml_dir + image[:-4] + '.xml')

['20190823_detection_result', '2019xxxx_detection_result', 'bak', 'best', 'record.txt', 'voc-vehicle-person-plate.data', 'yolov3-tiny-ncs-without-last-maxpool-refined-anchors-6cls-extend-remove-maxpooling_new_test.cfg', 'yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900.weights', 'yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_test.cfg', 'yolov3-tiny-ncs-without-last-maxpool-refined-anchors_112100.weights', 'yolov3-tiny-ncs-without-last-maxpool-refined-anchors_130000cars.weights', 'yolov3-tiny-ncs-without-last-maxpool-refined-anchors_test.cfg', 'yolov3_nob_official_test.cfg']
20190823_detection_result
2019xxxx_detection_result
bak
best
record.txt
voc-vehicle-person-plate.data
yolov3-tiny-ncs-without-last-maxpool-refined-anchors-6cls-extend-remove-maxpooling_new_test.cfg
yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900.weights
/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_test.cfg
Created : /media/yyp/DATA/EV

Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182005869753_vehicle_0000995.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182011886985_vehicle_0001034.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182016655259_vehicle_0001067.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182018556294_vehicle_0001078.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182022980114_vehicle_0001112.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182033289789_vehicle_0001203.xm

Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182336152588_vehicle_0003641.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182342681840_vehicle_0003853.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182344153979_vehicle_0003889.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182345102507_vehicle_0003921.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182347691105_vehicle_0003976.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182349103704_vehicle_0004003.xm

Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182546525480_vehicle_0006211.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182550109512_vehicle_0006253.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182552830986_vehicle_0006323.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182516744026_vehicle_0005703.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182527726271_vehicle_0005897.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182534545779_vehicle_0006037.xm

Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182729856922_vehicle_0008151.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182733392388_vehicle_0008204.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182733610382_vehicle_0008205.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182739646383_vehicle_0008302.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182741765311_vehicle_0008336.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/20190723182742528764_vehicle_0008351.xm

...

Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723181825558745_vehicle_0000113.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723181829822343_vehicle_0000137.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723181832210672_vehicle_0000157.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723181828416022_vehicle_0000126.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723181837079831_

Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182116629729_vehicle_0001550.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182118794166_vehicle_0001565.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182121796821_vehicle_0001607.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182124195956_vehicle_0001634.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182129196422_

Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182329979635_vehicle_0003482.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182340085822_vehicle_0003762.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182343710867_vehicle_0003879.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182323699869_vehicle_0003381.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182347112425_

Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182557680043_vehicle_0006395.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182601967776_vehicle_0006459.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182604119748_vehicle_0006527.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182604989442_vehicle_0006537.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182606710163_

Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182742528764_vehicle_0008351.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182744027777_vehicle_0008379.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182745705519_vehicle_0008414.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182729340755_vehicle_0008139.xml
Created : /media/yyp/DATA/EVALUATE/20190823_detection_result/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100900/ge_shrink_scale_0.2_0.2_127.5_fliped_20190723182731508595_